# Web Scrape Code

In [ ]:
from selenium.common.exceptions import NoSuchElementException

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

EPL_scrape = pd.DataFrame(columns=['home_possession', 'away_possession', 'home_passes', 'away_passes', 'home_offsides', 'away_offsides', 'attendance'])


# for loop to run through all games of 17/18 season 22342-22721 - 380 games
def scrape_possession_and_passes(dataframe):

    
    for num in range(22342, 22722):
        partial_url = 'https://www.premierleague.com/match/'
        url = partial_url + str(num)    
        driver.get(url)
        
        # let the page load correctly
        time.sleep(2)
        
        # click the stats tab
        
        try:
            elem1 = driver.find_element_by_xpath('//*[@id="mainContent"]/div/section/div[2]/div[2]/div[1]/div/div/ul/li[3]')
            elem1.click()
        except NoSuchElementException:  
            pass
        
        
        try:
            elem2 = driver.find_element_by_xpath('//*[@id="mainContent"]/div/section/div[2]/div/div[1]/div/div/ul/li[3]')
            elem2.click()
        except NoSuchElementException:  
            pass     
        
        
        
        try:
            elem3 = driver.find_element_by_xpath('//*[@id="mainContent"]/div/section/div[2]/div/div[1]/div/div/ul/li[3]')
            elem3.click()
        except NoSuchElementException as e: print(e, num)
            
            
        
            
        time.sleep(2)
        # things to scrape
        element = driver.find_element_by_xpath('//*[@id="mainContent"]/div/section/div[2]/div/div[2]/section[3]/div[2]/div[2]/table/tbody')
        full_table = element.find_elements_by_tag_name('p')
        
        home_possession = full_table[0].text
        away_possession = full_table[2].text
        
        home_passes = full_table[12].text
        away_passes = full_table[14].text
        
        home_offsides = full_table[24].text
        away_offsides = full_table[26].text
        
        crowd = driver.find_element_by_xpath('//*[@id="mainContent"]/div/section/div[2]/section/div[3]/div/div/div[2]/div[2]').text
        
        attendance = crowd[7:]
        
        
        
        #append each page results to dataframe
        dataframe = dataframe.append({'home_possession': str(home_possession), 'away_possession': str(away_possession), 'home_passes': str(home_passes), 'away_passes': str(away_passes), 'home_offsides': str(home_offsides), 'away_offsides': str(away_offsides), 'attendance': str(attendance)}, ignore_index=True)
        
        time.sleep(1)
        #print(url)
        #print(EPL_scrape.head())
    dataframe.to_csv('EPL_scrape_1718_test1.csv')
    return dataframe.shape

scrape_possession_and_passes(EPL_scrape)

# Code for Feature Engineering and Adding Web Scrape data to existing dataframe

In [ ]:
# add columns and change data set name to results

cols = ['Date', 'MatchDay', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HST', 'AST', 'Home_total_goals_for', 'Home_total_goals_conceded', 'Away_total_goals_for', 'Away_total_goals_conceded', 'home_goal_diff', 'away_goal_diff','HomeTeam_rank', 'AwayTeam_rank', 'Home_Attack', 'Away_Attack', 'Home_D', 'Away_D']

results = season_1718
results = results.reindex(columns=list(cols))

# add matchday values to data frame
for week in range(0,380):
    low = week*10
    high = low + 10
    match = week + 1
    
    for row in range(low, high):
        results.MatchDay.loc[row] = match
        
# give all teams rank of 1 to start the season
r = range(0, 10)
results.HomeTeam_rank.iloc[r] = int(1)
results.AwayTeam_rank.iloc[r] = int(1)

# add quick calculation of goal difference columns per game to allow rest of code to work for ranking
results.apply(lambda row: row.FTHG - row.FTAG, axis=1)
results['home_goal_diff'] = results.apply(lambda row: row.FTHG - row.FTAG, axis=1)

results.apply(lambda row: row.FTAG - row.FTHG, axis=1)
results['away_goal_diff'] = results.apply(lambda row: row.FTAG - row.FTHG, axis=1)

In [ ]:
# teams equal season.HomeTeam.unique
teams = ['Arsenal', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton', 'Southampton', 'Watford', 'West Brom', 'Man United', 'Newcastle', 'Bournemouth', 'Burnley', 'Leicester', 'Liverpool', 'Stoke', 'Swansea', 'Huddersfield', 'Tottenham', 'Man City', 'West Ham']
    
# create a dictionary to track table pts for team rank value
table_points = {}
for team in teams:
    if not team in table_points:
        table_points[team] = 0
    else:
        table_points[team] += 1
    
# create a dictionary for running goal difference to use as tiebreaker when table points are equal
running_goal_diff = {}
for team in teams:
    if not team in running_goal_diff:
        running_goal_diff[team] = 0
    else:
        running_goal_diff[team] += 1
            
            
# create a dictionary to track total goals for and total goals conceded for attack rank value
total_goals_for = {}
for team in teams:
    if not team in total_goals_for:
        total_goals_for[team] = 0
    else:
        total_goals_for[team] = 0
            

total_goals_conceded = {}
for team in teams:
    if not team in total_goals_conceded:
        total_goals_conceded[team] = 0
    else:
        total_goals_conceded[team] = 0

In [ ]:
import operator
from collections import defaultdict

for week in range(0, 39):
    low = week*10
    high = low + 10
    match = week + 1
    
    
    # update the league table by range of 10 games for each week - Table Points dictionary update
    for row in range(low, high):
        if results.FTR.iloc[row] == 'H':
            table_points[str(results.HomeTeam.iloc[row])] += 3
            table_points[str(results.AwayTeam.iloc[row])] += 0
        
        elif results.FTR.iloc[row] == 'A':
            table_points[str(results.AwayTeam.iloc[row])] += 3
            table_points[str(results.HomeTeam.iloc[row])] += 0

        elif results.FTR.iloc[row] == 'D':
            table_points[str(results.HomeTeam.iloc[row])] += 1
            table_points[str(results.AwayTeam.iloc[row])] += 1
        
        else:
            print('error in row', row)
            
    # update the goal differential in a new dictionary running_goal_diff
    for row in range(low,high):
        running_goal_diff[str(results.HomeTeam.iloc[row])] += results.home_goal_diff.iloc[row]
        running_goal_diff[str(results.AwayTeam.iloc[row])] += results.away_goal_diff.iloc[row]
    

    # combine the sorted dictionaries for one list of teams ranked by pts and goal diff
    weekly_pts = sorted(table_points.items(), key=operator.itemgetter(1), reverse=True)
    weekly_goal_diff = sorted(running_goal_diff.items(), key=operator.itemgetter(1), reverse=True)
    
    
    dd = defaultdict(list)

    for d in (table_points, running_goal_diff): 
        for key, value in d.items():
            dd[key].append(value)
    
    weekly_pts_goaldiff = sorted(dd.items(), key=operator.itemgetter(1), reverse=True)


    for row in range(low, high):
        home_team = results.HomeTeam.iloc[row]
        index_list = [x for x, y in enumerate(weekly_pts_goaldiff) if y[0] == home_team]
        results.HomeTeam_rank.iloc[row] = index_list[0] + 1
    
        away_team = results.AwayTeam.iloc[row]
        index_list = [x for x, y in enumerate(weekly_pts_goaldiff) if y[0] == away_team]
        results.AwayTeam_rank.iloc[row] = index_list[0] + 1
    
    for row in range(low, high):
    
        home_scored = results.FTHG.iloc[row]
        away_scored = results.FTAG.iloc[row]
    
        total_goals_for[str(results.HomeTeam.iloc[row])] += home_scored
        total_goals_for[str(results.AwayTeam.iloc[row])] += away_scored

        total_goals_conceded[str(results.HomeTeam.iloc[row])] += away_scored
        total_goals_conceded[str(results.AwayTeam.iloc[row])] += home_scored
    
    total_goals = defaultdict(list)

    for d in (total_goals_for, total_goals_conceded): 
        for key, value in d.items():
                total_goals[key].append(value)

    for row in range(low, high):
        home_team = results.HomeTeam.iloc[row]
        #index_list = [x for x, y in enumerate(total_goals) if y[0] == home_team]
        results.Home_total_goals_for.iloc[row] = total_goals[home_team][0]
        results.Home_total_goals_conceded.iloc[row] = total_goals[home_team][1]
        
        away_team = results.AwayTeam.iloc[row]
        #index_list = [x for x, y in enumerate(week1_pts_goaldiff) if y[0] == away_team]
        results.Away_total_goals_for.iloc[row] = total_goals[away_team][0]
        results.Away_total_goals_conceded.iloc[row] = total_goals[away_team][1]

In [ ]:
# add spread for each game
results.apply(lambda row: row.FTHG - row.FTAG, axis=1)
results['Spread'] = results.apply(lambda row: row.FTHG - row.FTAG, axis=1)

# home attack and away attack code

home_goal_agg_mean = 1.5403508771929826
away_goal_agg_mean = 1.1859649122807017

home_conceded_agg_mean = away_goal_agg_mean
away_conceded_agg_mean = home_goal_agg_mean


for row in range(0, 380):
    results.Home_Attack[row] = ((results.Home_total_goals_for[row] / results.MatchDay[row]) / home_goal_agg_mean)
    results.Away_Attack[row] = ((results.Away_total_goals_for[row] / results.MatchDay[row]) / away_goal_agg_mean)
    
    results.Home_D[row] = ((results.Home_total_goals_conceded[row] / results.MatchDay[row]) / home_conceded_agg_mean)
    results.Away_D[row] = ((results.Away_total_goals_conceded[row] / results.MatchDay[row]) / away_conceded_agg_mean)




# Code for Modeling

In [5]:
%pylab inline
#% config InlineBackend.figure_format = 'svg'

import pandas as pd
import seaborn as sns
import numpy as np
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

Populating the interactive namespace from numpy and matplotlib


In [42]:
data = pd.read_csv('1516_1617_1718.csv')

In [43]:
small_data = data.loc[:,['A_Promotion', 'HST', 'AST', 'H_vs_A_goal_diff_season', 'H_Fifa_ranked',
    'Home_Attack', 'Away_Attack', 'Home_D', 'Away_D', 'H_A_Wage_Diff', 'Spread']]

In [95]:
X = small_data.loc[:,['A_Promotion', 'HST', 'AST', 'H_vs_A_goal_diff_season', 'H_Fifa_ranked',
    'Home_Attack', 'Away_Attack', 'Home_D', 'Away_D', 'H_A_Wage_Diff']]

y = small_data['Spread']

# OLS    .50 R^2 on training data

In [ ]:
y, X = patsy.dmatrices('Spread ~ H_Promotion + A_Promotion + HST + AST + H_vs_A_goal_diff_season + H_Fifa_ranked + A_Fifa_ranked + HomeTeam_rank + AwayTeam_rank + Rank_ratio + Home_Attack + Away_Attack + Home_D + Away_D', data=smaller_data, return_type="dataframe")


model = sm.OLS(y, X)
fit = model.fit()
fit.summary()

# Lasso and Ridge 

In [96]:
selected_columns = ['A_Promotion', 'HST', 'AST', 'H_vs_A_goal_diff_season', 'H_Fifa_ranked', 'Home_Attack', 'Away_Attack', 'Home_D', 'Away_D', 'H_A_Wage_Diff']

In [125]:
#Split the data 60 - 20 - 20 train/val/test

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=19)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=19)

In [126]:
# apply the scaler
std = StandardScaler()
std.fit(X_train_val.values)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [127]:
# transform with scaler
X_tr = std.transform(X_train_val.values)
X_te = std.transform(X_test.values)

## Lasso w 10 features + scaler transform   R^2 .523   RMSE 1.324

In [119]:
alphavec = 10**np.linspace(-2,2,200)

lasso = LassoCV(alphas = alphavec, cv=5)
lasso.fit(X_tr, y_train_val)

LassoCV(alphas=array([1.00000e-02, 1.04737e-02, ..., 9.54772e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [120]:
lasso.alpha_

0.015885651294280528

In [121]:
test_set_pred = lasso.predict(X_te)

In [122]:
r2_score(y_test, test_set_pred)

0.4720989586458376

In [123]:
p = test_set_pred
err = abs(p-y_test)
total_err = np.dot(err,err)
lasso_rmse = np.sqrt(total_err/len(p))
print(lasso_rmse)

1.3340790894482488


## Ridge w 10 features + scaler transform   R^2 .523   RMSE 1.324

In [128]:
alphavec = 10**np.linspace(-2,2,200)

Ridge_model = RidgeCV(alphas=alphavec, cv=5)
Ridge_model.fit(X_tr, y_train_val)

RidgeCV(alphas=array([1.00000e-02, 1.04737e-02, ..., 9.54772e+01, 1.00000e+02]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [129]:
Ridge_model.alpha_

43.470131581250264

In [130]:
test_set_pred = lasso.predict(X_te)

In [131]:
r2_score(y_test, test_set_pred)

0.4720989586458376

In [132]:
p = test_set_pred
err = abs(p-y_test)
total_err = np.dot(err,err)
ridge_rmse = np.sqrt(total_err/len(p))
print(ridge_rmse)

1.3340790894482488
